In [94]:
import pandas as pd

In [95]:
df_all = pd.read_csv('../data/df_all.csv').drop(['Unnamed: 0', 'id', 'cluster', 'sentiment'], axis=1)

In [96]:
df_all

,question,answer,count
0,Что вы сможете использовать в своей работе?,"формулировать проблему, 5 поче",1.0
1,Что вы сможете использовать в своей работе?,поиск потерь,1.0
2,Что вы сможете использовать в своей работе?,правило 5 «почему» :),1.0
3,Что вы сможете использовать в своей работе?,умение работать в команде,1.0
4,Что вы сможете использовать в своей работе?,картирование и анализ,1.0
...,...,...,...
29659,•Где вы хотите провести следующую HR-сессию?,в москве,1.0
29660,•Где вы хотите провести следующую HR-сессию?,в сочи,1.0
29661,•Где вы хотите провести следующую HR-сессию?,сочи,1.0
29662,•Где вы хотите провести следующую HR-сессию?,окбм,1.0


In [97]:
index = pd.MultiIndex.from_arrays(df_all.iloc[:, :-1].values.T, names=('question', 'answer'))
df_grouped = pd.DataFrame({'count' : df_all.iloc[:, -1:].values.T[0]}, index=index)

In [98]:
df_grouped

count
question                                     answer                               
Что вы сможете использовать в своей работе?  формулировать проблему, 5 поче    1.0
                                             поиск потерь                      1.0
                                             правило 5 «почему» :)             1.0
                                             умение работать в команде         1.0
                                             картирование и анализ             1.0
...                                                                            ...
•Где вы хотите провести следующую HR-сессию? в москве                          1.0
                                             в сочи                            1.0
                                             сочи                              1.0
                                             окбм                              1.0
                                             сочи                              1.0

[29664 rows x 1 columns]

In [112]:
abbreviations = dict()
with open('../data/слова.txt') as file:
    for line in file.readlines():
        key, value = map(str.strip, line.split('->'))
        abbreviations[key] = value

In [113]:
abbreviations

{'гуд': 'хорошо',
 '[норм, норма]': 'нормально',
 'бюрокр': 'бюрократия',
 'ис': 'информационная система',
 'по': 'программное обеспечение',
 'нд': 'нормативные документы',
 '[доку, док, док-тов, доки, док]': 'документы',
 'еос': 'единая отраслевая система',
 'еосдо': 'единая отраслевая система документооборота',
 'еосз': 'единая отраслевая система закупок',
 'эл.': 'электронный',
 'лна': 'локальный нормативный акт',
 'тс уп': 'типовая система управление персоналом',
 'удаленка': 'удаленная работа',
 'прогр.': 'программа',
 'быстродейств.': 'быстродействие',
 'plannera': 'планировщик',
 'кол-во': 'количество',
 '[раб, раб.]': 'работа',
 'ответс-ти': 'ответственность',
 '[it, айти]': 'информационные технологии',
 '[кринжовое, кринж, кринжую]': 'плохо',
 '[чильное, чил, чилл, бест, за бест, зе бест, кайф]': 'отлично',
 '[не оч, неоч, не очень]': 'грустно'}